In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Notes from RT Kickoff Meeting

* Incorporating trip updates?
* Trip updates can include a measure of delay…
* Relying more on internal processes…
* Trip updates also include overall schedule deviation
* Data can be missing from RT: vehicles without trips, no delay, etc…
* Evan: vehicle positions to infer delay if trip updates available, we’re implicitly deciding that we don’t trust trip updates → trusting the schedule and vehicle positions 100%
* We have trip updates someplace, just needs to be pipelined…
* In theory, update frequency should keep positions and updates close?
* Some feeds omit past events once a bus passes stop, or even future events)
* Data quality:
    * Every 20 seconds, look at next arrival from trip updates…
    * Exploratory analysis? Quicker to use+trust trip updates…


* Analysis timespan
* Confirming agency counts

#### Action items

* Work towards loading trip updates into warehouse
* Look into why there are fewer RT agencies in warehouse (~16) compared to agencies.yml (~30)
* Initial work on an algorithm to interpolate vehicle positions
* Assess how different various feeds’ implementations of trip updates are


In [2]:
## set system time

os.environ['TZ'] = 'America/Los_Angeles'
time.tzset()
time.tzname

('PST', 'PDT')

In [3]:
lbt_itp_id = 170

In [4]:
# %%sql -o lbt_positions

# # get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
# SELECT *
# FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
# WHERE _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-02_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-03_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-04_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-05_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-06_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-07_170_0.parquet"
# ORDER BY header_timestamp

In [5]:
# lbt_positions.to_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [6]:
lbt_positions = pd.read_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [7]:
def convert_ts(ts):    
    pacific_dt = dt.datetime.fromtimestamp(ts)
    # print(pacific_dt)
    return pacific_dt

In [8]:
lbt_positions.vehicle_timestamp = lbt_positions.vehicle_timestamp.apply(convert_ts)
lbt_positions.header_timestamp = lbt_positions.header_timestamp.apply(convert_ts)

In [9]:
min_date, max_date = ('2021-11-01', '2021-12-01')
pch_routes = ('171', '172', '173', '174', '175')

In [10]:
# lbt_trips = (tbl.views.gtfs_schedule_fact_daily_trips()
#     # >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
#     >> filter(_.calitp_itp_id == lbt_itp_id)
#     >> filter(_.service_date == max_date)
#     >> filter(_.is_in_service == True)
#     >> select(_.trip_key, _.service_date)
#     >> inner_join(_, tbl.views.gtfs_schedule_dim_trips(), on = 'trip_key')
#     >> select(_.calitp_itp_id, _.calitp_url_number,
#               _.date == _.service_date,
#               _.trip_key, _.trip_id, _.route_id, _.direction_id,
#               _.shape_id, _.calitp_extracted_at, _.calitp_deleted_at)
#     # >> filter(_.route_id.isin(pch_routes))

#     >> collect()
#     )

In [11]:
# lbt_trips.to_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [12]:
lbt_trips = pd.read_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [13]:
st = (tbl.gtfs_schedule.stop_times()
     >> filter(_.calitp_itp_id == 170)
     # >> filter(_.trip_id == '9288623')
     >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
     >> arrange(_.stop_sequence)
     >> collect()
    )

In [14]:
stops = (tbl.gtfs_schedule.stops()
     >> filter(_.calitp_itp_id == 170)
     # >> filter(_.trip_id == '9288623')
     # >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
     # >> arrange(_.stop_sequence)
     >> select(_.calitp_itp_id, _.calitp_url_number, _.stop_id,
              _.stop_lat, _.stop_lon, _.stop_name)
     >> collect()
    )

stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
                        crs='EPSG:4326').to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [15]:
trips_positions_joined = (lbt_trips
                        >> filter(_.route_id.isin(pch_routes))
                        >> inner_join(_, (lbt_positions >> rename(trip_id = 'vehicle_trip_id')),
                                      on= ['trip_id', 'calitp_itp_id', 'calitp_url_number'])
                       )

In [16]:
trips_positions_joined.vehicle_timestamp.max()

Timestamp('2021-12-01 11:15:58')

In [17]:
trips_positions_joined.vehicle_timestamp.min()

Timestamp('2021-11-30 15:56:12')

In [18]:
trips_positions_joined = gpd.GeoDataFrame(trips_positions_joined,
                                geometry=gpd.points_from_xy(trips_positions_joined.vehicle_position_longitude,
                                                            trips_positions_joined.vehicle_position_latitude),
                                crs=shared_utils.geography_utils.WGS84).to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [19]:
lbt_routelines = shared_utils.geography_utils.make_routes_shapefile(['170'], ## LBT
                                shared_utils.geography_utils.CA_NAD83Albers)

In [20]:
MPH_PER_MPS = 2.237

In [41]:
class VehiclePositionsTrip:
    '''Trip data and useful methods for analyzing GTFS-RT vehicle positions data'''
    
    def attach_shape(self, shape_gdf):
        assert shape_gdf.crs == shared_utils.geography_utils.CA_NAD83Albers
        assert shape_gdf.calitp_itp_id.iloc[0] == self.calitp_itp_id
        
        self.shape = (shape_gdf
                        >> filter(_.shape_id == self.shape_id)
                        >> select(_.shape_id, _.geometry))
        self.linear_reference()
    
    def __init__(self, vp_gdf, shape_gdf):
        assert vp_gdf.crs == shared_utils.geography_utils.CA_NAD83Albers
        vp_gdf = vp_gdf >> distinct(_.trip_id, _.vehicle_timestamp, _keep_all=True)
        
        self.date = vp_gdf.date.iloc[0]
        self.trip_id = vp_gdf.trip_id.iloc[0]
        self.route_id = vp_gdf.route_id.iloc[0]
        self.shape_id = vp_gdf.shape_id.iloc[0]
        self.entity_id = vp_gdf.entity_id.iloc[0]
        self.vehicle_id = vp_gdf.vehicle_id.iloc[0]
        self.calitp_itp_id = vp_gdf.calitp_itp_id.iloc[0]
        self.calitp_url_number = vp_gdf.calitp_url_number.iloc[0]
        self.vehicle_positions = vp_gdf >> select(_.vehicle_timestamp,
                                              _.header_timestamp,
                                              _.geometry)
        self.attach_shape(shape_gdf)
        
    def linear_reference(self):
        self.vehicle_positions['shape_meters'] = (self.vehicle_positions.geometry
                                        .apply(lambda x: self.shape.geometry.iloc[0].project(x)))
        self.vehicle_positions['last_time'] = self.vehicle_positions.vehicle_timestamp.shift(1)
        self.vehicle_positions['last_loc'] = self.vehicle_positions.shape_meters.shift(1)
        self.vehicle_positions['secs_from_last'] = self.vehicle_positions.vehicle_timestamp - self.vehicle_positions.last_time
        self.vehicle_positions.secs_from_last = (self.vehicle_positions.secs_from_last
                                        .apply(lambda x: x.seconds))
        self.vehicle_positions['meters_from_last'] = (self.vehicle_positions.shape_meters
                                                      - self.vehicle_positions.last_loc)
        self.vehicle_positions['progressed'] = self.vehicle_positions['meters_from_last'] > 0 ## has the bus moved ahead?
        self.vehicle_positions['speed_from_last'] = (self.vehicle_positions.meters_from_last
                                                     / self.vehicle_positions.secs_from_last)
    

        
#     def position_at_time(self, dt):
        
    def time_at_position(self, desired_position):
        
        global bounding_points
        
        try:
            next_point = (self.vehicle_positions
                  >> filter(_.progressed)
                  >> filter(_.shape_meters > desired_position)
                  >> filter(_.shape_meters == _.shape_meters.min())
                 )
            prev_point = (self.vehicle_positions
                  >> filter(_.progressed)
                  >> filter(_.shape_meters < desired_position)
                  >> filter(_.shape_meters == _.shape_meters.max())
                 )
            bounding_points = (prev_point.append(next_point).copy().reset_index(drop=True)
                    >> select(-_.secs_from_last, -_.meters_from_last, -_.speed_from_last)) ## need to drop in case bounding points are nonconsecutive
            secs_from_last = (bounding_points.loc[1].vehicle_timestamp - bounding_points.loc[0].vehicle_timestamp).seconds
            meters_from_last = bounding_points.loc[1].shape_meters - bounding_points.loc[0].shape_meters
            speed_from_last = meters_from_last / secs_from_last

            meters_position_to_next = bounding_points.loc[1].shape_meters - desired_position
            est_seconds_to_next = meters_position_to_next / speed_from_last
            est_td_to_next = dt.timedelta(seconds=est_seconds_to_next)
            est_dt = bounding_points.iloc[-1].vehicle_timestamp - est_td_to_next

            return est_dt
        except KeyError:
            print(f'insufficient bounding points for location {desired_position}')
            print(f'start/end of route?')
            return None

In [42]:
def delay_view(trip_id, trips_positions_joined, stop_times, stops, shape_gdf):
    global _debug
    global trip_rt_data
    
    this_trip = trips_positions_joined >> filter(_.trip_id == trip_id)
    trip_rt_data = VehiclePositionsTrip(this_trip, shape_gdf)
    
    trip_st = stop_times >> filter(_.trip_id == trip_id)
    trip_st_geo = stops >> inner_join(_, trip_st, on = ['calitp_itp_id', 'calitp_url_number',
                                     'stop_id'])
    trip_st_geo['linear_meters'] = (trip_st_geo.geometry
                                        .apply(lambda x: trip_rt_data.shape.project(x)))
    trip_st_geo['actual_time'] = trip_st_geo['linear_meters'].apply(lambda x: trip_rt_data.time_at_position(x))
    trip_st_geo['arrival_time'] = trip_st_geo.arrival_time.apply(lambda x:
                                    dt.datetime.combine(trip_st_geo.actual_time.iloc[0].date(),
                                                        dt.datetime.strptime(x, '%H:%M:%S').time()))
    _debug = trip_st_geo
    trip_st_geo['delay'] = trip_st_geo.actual_time - trip_st_geo.arrival_time
    trip_st_geo['date'] = trip_rt_data.date
    trip_view = trip_st_geo.dropna(subset=['delay']) >> arrange(_.arrival_time) >> select(
                                                    _.arrival_time, _.actual_time, _.delay,
                                                 _.stop_id, _.trip_id, _.stop_sequence,
                                            _.date, _.geometry)
    return trip_view

In [43]:
# delay_view('9288623', trips_positions_joined, st, stops, lbt_routelines)

In [44]:
# _debug

In [45]:
example_trips = (trips_positions_joined
 >> filter(_.route_id == '175', _.direction_id == '0') ## towards Villages at Cabrillo
 >> group_by(_.trip_id)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
)
example_trips.head(6)

,trip_id,max_time
8,9288619,2021-11-30 16:37:23
9,9288621,2021-11-30 17:17:58
10,9288623,2021-11-30 17:56:04
11,9288625,2021-11-30 18:35:49
12,9288627,2021-11-30 19:24:12
13,9288629,2021-11-30 19:59:08


In [46]:
example_trips.trip_id.iloc[:6]

8     9288619
9     9288621
10    9288623
11    9288625
12    9288627
13    9288629
Name: trip_id, dtype: object

In [47]:
delays = gpd.GeoDataFrame()

for trip_id in example_trips.trip_id.iloc[:6]:
    
    single_trip_delay = delay_view(trip_id, trips_positions_joined, st, stops, lbt_routelines)
    delays = delays.append(single_trip_delay)

insufficient bounding points for location 11480.641185357437
start/end of route?
insufficient bounding points for location 991.1092912035456
start/end of route?
insufficient bounding points for location 1666.4024361313238
start/end of route?
insufficient bounding points for location 501.0407611454837
start/end of route?
insufficient bounding points for location 0.0
start/end of route?
insufficient bounding points for location 501.0407611454837
start/end of route?
insufficient bounding points for location 0.0
start/end of route?
insufficient bounding points for location 11480.641185357437
start/end of route?
insufficient bounding points for location 0.0
start/end of route?
insufficient bounding points for location 11480.641185357437
start/end of route?
insufficient bounding points for location 0.0
start/end of route?
insufficient bounding points for location 0.0
start/end of route?
insufficient bounding points for location 11480.641185357437
start/end of route?
insufficient bounding poi

In [50]:
delays

,arrival_time,actual_time,delay,stop_id,trip_id,stop_sequence,date,geometry
15,2021-11-30 15:43:27,2021-11-30 16:01:08.496330,0 days 00:17:41.496330,0543,9288619,5,2021-12-01,POINT (173108.288 -468477.715)
26,2021-11-30 15:44:00,2021-11-30 16:01:37.389317,0 days 00:17:37.389317,0544,9288619,6,2021-12-01,POINT (173001.551 -468534.279)
28,2021-11-30 15:45:15,2021-11-30 16:03:06.190004,0 days 00:17:51.190004,4054,9288619,7,2021-12-01,POINT (172801.703 -468384.262)
16,2021-11-30 15:46:23,2021-11-30 16:03:44.801889,0 days 00:17:21.801889,4055,9288619,8,2021-12-01,POINT (172606.266 -468217.848)
21,2021-11-30 15:48:00,2021-11-30 16:04:26.802384,0 days 00:16:26.802384,1346,9288619,9,2021-12-01,POINT (172322.827 -467985.831)
...,...,...,...,...,...,...,...,...
14,2021-11-30 19:32:08,2021-11-30 19:33:19.123625,0 days 00:01:11.123625,0011,9288629,24,2021-12-01,POINT (166864.163 -467876.750)
8,2021-11-30 19:32:42,2021-11-30 19:33:50.098569,0 days 00:01:08.098569,0012,9288629,25,2021-12-01,POINT (166631.631 -467882.275)
17,2021-11-30 19:35:01,2021-11-30 19:35:09.357729,0 days 00:00:08.357729,3140,9288629,26,2021-12-01,POINT (165660.454 -467899.210)
10,2021-11-30 19:36:00,2021-11-30 19:37:09.103320,0 days 00:01:09.103320,3048,9288629,27,2021-12-01,POINT (165247.302 -467911.875)


In [ ]:
# delays >> group_by(_.stop_id) >> summarize(avg_delay = _.delay.mean()) >> inner_join(_, delays, on = 'stop_id')

In [ ]:
test1 = (delays >> group_by(_.stop_id) >> summarize(max_delay = _.delay.max()) >> inner_join(_, delays, on = 'stop_id'))

In [ ]:
test1 >> filter(_.delay > dt.timedelta(hours=1))

In [ ]:
# example_trip = delays >> filter(_.trip_id == '9288621')
example_trip = delay_view('9288623', trips_positions_joined, st, stops, lbt_routelines)
example_trip['delay_minutes'] = example_trip.delay.apply(lambda x: round((x.seconds / 60), 0))

In [ ]:
# simple_map(example_trip >> select(_.geometry, _.delay_minutes), 'delay_minutes')

In [ ]:
buggy_trip = delay_view('9288627', trips_positions_joined, st, stops, lbt_routelines)
## actually not a bug, early arrivals wrap around...
-buggy_trip.delay.iloc[0]

In [ ]:
## a good view: trip, (direction), stop, timedelta?
## plot deltas with a sequential scheme...

In [ ]:
delay_view('9288625', trips_positions_joined, st, stops, lbt_routelines);

In [ ]:
trip_rt_data.time_at_position(3309)

In [ ]:
trip_rt_data.time_at_position(4309)

In [ ]:
trip_rt_data.time_at_position(5309)

In [ ]:
bounding_points

In [ ]:
_debug